# <strong>5: ELAN Annotation Notebook |</strong>


### <strong>Requirements</strong>

To run this notebook, you will need the following Python packages:

- pandas

In [ ]:
import os
import pandas as pd

In [ ]:
input_directory = '/n/kfranich_speech_l3/Lab/6000_EMA/processed_articulatory_data/video_processing/final_apexes/SN6014_ARTGEST_ENG_VIDAUD/'  # Replace with your directory path
output_directory = '/n/kfranich_speech_l3/Lab/6000_EMA/processed_articulatory_data/video_processing/elan_importable_annotations/SN6014_ARTGEST_ENG/'  # Replace with your desired output directory

### <strong>Export Apex Annotations</strong>

In [ ]:
# Iterate through all CSV files in the directory
for filename in os.listdir(input_directory):
    if filename.endswith('.csv'):
        # Read the CSV file
        file_path = os.path.join(input_directory, filename)
        apex_df = pd.read_csv(file_path)

        apex_df['change_points'] = apex_df['apex'].shift(1) != apex_df['apex']
        apex_df['time_ms'] = apex_df['time_ms'] / 1000
        grouped_apex_df = apex_df.groupby((apex_df['change_points']).cumsum())
        apex_df = pd.DataFrame({
            'Begin Time': grouped_apex_df['time_ms'].first(),
            'End Time': grouped_apex_df['time_ms'].last(),
            'apex': grouped_apex_df['apex'].first(),
        })
        apex_df = apex_df.dropna(subset=['apex'])
        apex_df = apex_df[apex_df['apex'] != 0]
        apex_df['End Time'] = apex_df['End Time'] + 0.033

        # Save the modified DataFrame
        output_file_path = os.path.join(output_directory, filename.split('.')[0] + '_elan_apexes.csv')
        apex_df.to_csv(output_file_path, index=False)

In [ ]:
apex_df.head(20)